In [11]:
API_KEY = "4bfd24cf3b2e591c5246a66c832ce9d6"
BASE_URL = "http://ws.audioscrobbler.com/2.0/"
API_SECRET = "2020b233790a14f8433e32628ecfa78c"

In [12]:
import requests
import hashlib

In [13]:
def generate_signature(params):
    """
    Generate API signature by concatenating sorted params and appending API_SECRET.
    """
    # Concatenate parameters in alphabetical order
    signature_string = "".join(f"{key}{value}" for key, value in sorted(params.items()))
    # Append API_SECRET to the string
    signature_string += API_SECRET
    # Return the MD5 hash
    return hashlib.md5(signature_string.encode("utf-8")).hexdigest()

In [14]:
def get_token():
    """
    Request an authentication token from Last.fm with a signed API call.
    """
    # Define the parameters for the request
    params = {
        "method": "auth.getToken",
        "api_key": API_KEY,
    }

    # Generate the API signature
    params["api_sig"] = generate_signature(params)
    params["format"] = "json"  # Add format separately as it is not part of the signature

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        return data["token"]
    else:
        raise Exception(f"Error getting token: {response.status_code}, {response.text}")


In [31]:
# Example usage
token = get_token()
print(f"Token: {token}")

Token: KN-1jTI3iBKzS-QJWrxZ3kVDSR9a2FoS


In [16]:
def get_artist_info(artist_name):
    params = {
        "method": "artist.getinfo",  # API method
        "artist": artist_name,       # Artist name to search
        "api_key": API_KEY,          # Your Last.fm API key
        "format": "json"             # Response format
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    # Check for successful response
    if response.status_code == 200:
        data = response.json()
        if "artist" in data:
            return data["artist"]
        else:
            return f"Error: {data.get('message', 'Unknown error')}"
    else:
        return f"HTTP Error: {response.status_code}"

In [17]:
artist_name = "Coldplay"
artist_info = get_artist_info(artist_name)

if isinstance(artist_info, dict):
    print(f"Artist: {artist_info['name']}")
    print(f"Listeners: {artist_info['stats']['listeners']}")
    print(f"Playcount: {artist_info['stats']['playcount']}")
else:
    print(artist_info)

Artist: Coldplay
Listeners: 7842785
Playcount: 604595777


In [39]:
def get_top_tracks_by_country(country_code, token):
    """
    Get the top 50 tracks from a specified country using the Last.fm API.

    Args:
        country_code (str): ISO 3166-1 alpha-2 country code (e.g., "US" for United States).
        token (str): Authorized token obtained from Last.fm authentication.

    Returns:
        list: A list of dictionaries containing the rank, track name, artist, and playcount.
    """
    # Define the parameters for the request
    params = {
        "method": "geo.gettoptracks",
        "country": country_code,
        "api_key": API_KEY,
        "token": token,
        "limit": 50,  # Maximum number of tracks
    }

    # Generate the API signature
    params["api_sig"] = generate_signature(params)
    params["format"] = "xml"  # Request XML response format

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        from xml.etree import ElementTree as ET

        # Parse the XML response
        root = ET.fromstring(response.content)
        tracks = []

        for track in root.findall("track"):
            print('hi')
            tracks.append({
                "rank": track.get("rank"),
                "name": track.find("name").text,
                "artist": track.find("artist/name").text,
                "playcount": track.find("playcount").text,
                "url": track.find("url").text
            })

        return tracks
    else:
        raise Exception(f"Error getting top tracks: {response.status_code}, {response.text}")


In [40]:
try:
    country_code = "Brazil"
    top_tracks = get_top_tracks_by_country(country_code, token)
    print(top_tracks)
    for i, track in enumerate(top_tracks, start=1):
        print(f"{i}. {track['name']} by {track['artist']}")
except Exception as e:
    print(e)

[]
